In [1]:
import findspark
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import  KMeans
from pyspark.sql import SQLContext, Window
from pyspark.sql.functions import lit, row_number
import pandas as pd
import glob
import yaml
import csv
import numpy as np

In [2]:
spark = SparkSession.builder.appName('data_analysis').getOrCreate()
matches = spark.read .format("csv").option("header", "true").load("matches_2019.csv")
deliveries = spark.read .format("csv").option("header", "true").load("deliveries_2019.csv")

In [3]:
matches.printSchema()

root
 |-- id: string (nullable = true)
 |-- season: string (nullable = true)
 |-- city: string (nullable = true)
 |-- date: string (nullable = true)
 |-- team1: string (nullable = true)
 |-- team2: string (nullable = true)
 |-- toss_winner: string (nullable = true)
 |-- toss_decision: string (nullable = true)
 |-- result: string (nullable = true)
 |-- dl_applied: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- win_by_runs: string (nullable = true)
 |-- win_by_wickets: string (nullable = true)
 |-- player_of_match: string (nullable = true)
 |-- venue: string (nullable = true)
 |-- umpire1: string (nullable = true)
 |-- umpire2: string (nullable = true)
 |-- umpire3: string (nullable = true)



In [4]:
deliveries.printSchema()

root
 |-- match_id: string (nullable = true)
 |-- inning: string (nullable = true)
 |-- batting_team: string (nullable = true)
 |-- bowling_team: string (nullable = true)
 |-- over: string (nullable = true)
 |-- ball: string (nullable = true)
 |-- batsman: string (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- is_super_over: string (nullable = true)
 |-- wide_runs: string (nullable = true)
 |-- bye_runs: string (nullable = true)
 |-- legbye_runs: string (nullable = true)
 |-- noball_runs: string (nullable = true)
 |-- penalty_runs: string (nullable = true)
 |-- batsman_runs: string (nullable = true)
 |-- extra_runs: string (nullable = true)
 |-- total_runs: string (nullable = true)
 |-- player_dismissed: string (nullable = true)
 |-- dismissal_kind: string (nullable = true)
 |-- fielder: string (nullable = true)



In [5]:
matches.createOrReplaceTempView("matches")
deliveries.createOrReplaceTempView("deliveries")

In [6]:
df1 = spark.sql("select match_id, batting_team, sum(total_runs)/count(dismissal_kind) as Runs_per_Wicket from deliveries group by match_id, batting_team order by cast(match_id as int)")

In [7]:
df1.show()

+--------+--------------------+------------------+
|match_id|        batting_team|   Runs_per_Wicket|
+--------+--------------------+------------------+
|       1|Royal Challengers...|              17.2|
|       1| Sunrisers Hyderabad|             51.75|
|       2|      Mumbai Indians|              23.0|
|       2|Rising Pune Super...|62.333333333333336|
|       3|       Gujarat Lions|             45.75|
|       3|Kolkata Knight Ri...|              null|
|       4|     Kings XI Punjab|              41.0|
|       4|Rising Pune Super...|27.166666666666668|
|       5|    Delhi Daredevils|15.777777777777779|
|       5|Royal Challengers...|            19.625|
|       6|       Gujarat Lions|19.285714285714285|
|       6| Sunrisers Hyderabad|             140.0|
|       7|      Mumbai Indians|              30.0|
|       7|Kolkata Knight Ri...|25.428571428571427|
|       8|     Kings XI Punjab|              75.0|
|       8|Royal Challengers...|              37.0|
|       9|    Delhi Daredevils|

In [8]:
df1= df1.withColumn("new_column",lit("ABC"))
w = Window().partitionBy('new_column').orderBy(lit('A'))
df1 = df1.withColumn("row_num", row_number().over(w)).drop("new_column")

In [9]:
df1.show()

+--------+--------------------+------------------+-------+
|match_id|        batting_team|   Runs_per_Wicket|row_num|
+--------+--------------------+------------------+-------+
|       1| Sunrisers Hyderabad|             51.75|      1|
|       1|Royal Challengers...|              17.2|      2|
|       2|      Mumbai Indians|              23.0|      3|
|       2|Rising Pune Super...|62.333333333333336|      4|
|       3|       Gujarat Lions|             45.75|      5|
|       3|Kolkata Knight Ri...|              null|      6|
|       4|     Kings XI Punjab|              41.0|      7|
|       4|Rising Pune Super...|27.166666666666668|      8|
|       5|Royal Challengers...|            19.625|      9|
|       5|    Delhi Daredevils|15.777777777777779|     10|
|       6| Sunrisers Hyderabad|             140.0|     11|
|       6|       Gujarat Lions|19.285714285714285|     12|
|       7|      Mumbai Indians|              30.0|     13|
|       7|Kolkata Knight Ri...|25.428571428571427|     1

In [10]:
df1.createOrReplaceTempView("df1")

In [11]:
row_odd = spark.sql('select * from df1 where row_num%2 !="0"')
row_even = spark.sql('select * from df1 where row_num%2 ="0"')

In [12]:
row_odd.createOrReplaceTempView("row_odd")
row_even.createOrReplaceTempView("row_even")

In [13]:
rpw_odd_row = spark.sql("select row_odd.match_id, row_odd.batting_team, row_odd.Runs_per_Wicket/row_even.Runs_per_Wicket as Runs_per_Wicket_Ratio from row_odd join row_even where row_odd.match_id = row_even.match_id")

In [14]:
rpw_even_row = spark.sql("select row_even.match_id, row_even.batting_team, row_even.Runs_per_Wicket/row_odd.Runs_per_Wicket as Runs_per_Wicket_Ratio from row_odd join row_even where row_odd.match_id = row_even.match_id")

In [15]:
rpw_odd_row.createOrReplaceTempView("rpw_odd")
rpw_even_row.createOrReplaceTempView("rpw_even")

In [16]:
rpw_odd_row.count()

754

In [17]:
rpw_even_row.count()

754

In [18]:
rpw_ratio = spark.sql("select rpw_odd.match_id, rpw_odd.batting_team as team_1, rpw_odd.Runs_per_Wicket_Ratio, rpw_even.batting_team as team_2, rpw_even.Runs_per_Wicket_Ratio from rpw_odd join rpw_even where rpw_odd.match_id = rpw_even.match_id order by cast(match_id as int)")

# Runs per Wicket Ratio for every team in every match played

In [19]:
rpw_ratio.show()

+--------+--------------------+---------------------+--------------------+---------------------+
|match_id|              team_1|Runs_per_Wicket_Ratio|              team_2|Runs_per_Wicket_Ratio|
+--------+--------------------+---------------------+--------------------+---------------------+
|       1| Sunrisers Hyderabad|   3.0087209302325584|Royal Challengers...|   0.3323671497584541|
|       2|      Mumbai Indians|   0.3689839572192513|Rising Pune Super...|    2.710144927536232|
|       3|       Gujarat Lions|                 null|Kolkata Knight Ri...|                 null|
|       4|     Kings XI Punjab|     1.50920245398773|Rising Pune Super...|   0.6626016260162602|
|       5|Royal Challengers...|    1.243838028169014|    Delhi Daredevils|   0.8039631988676575|
|       6| Sunrisers Hyderabad|   7.2592592592592595|       Gujarat Lions|   0.1377551020408163|
|       7|      Mumbai Indians|   1.1797752808988764|Kolkata Knight Ri...|   0.8476190476190476|
|       8|Royal Challengers...

In [20]:
rpw_ratio.createOrReplaceTempView("rpw_ratio")

In [21]:
rr = spark.sql("select match_id, batting_team, sum(total_runs)/(count(ball)/6) as Runs_Rate from deliveries group by match_id, batting_team order by cast(match_id as int)")

In [22]:
rr.show()

+--------+--------------------+------------------+
|match_id|        batting_team|         Runs_Rate|
+--------+--------------------+------------------+
|       1|Royal Challengers...| 8.390243902439025|
|       1| Sunrisers Hyderabad|             9.936|
|       2|      Mumbai Indians|             8.832|
|       2|Rising Pune Super...| 9.196721311475411|
|       3|Kolkata Knight Ri...|              11.5|
|       3|       Gujarat Lions|               9.0|
|       4|Rising Pune Super...| 7.824000000000001|
|       4|     Kings XI Punjab| 8.065573770491804|
|       5|Royal Challengers...| 7.596774193548387|
|       5|    Delhi Daredevils| 6.870967741935483|
|       6| Sunrisers Hyderabad|              8.75|
|       6|       Gujarat Lions|              6.75|
|       7|      Mumbai Indians| 8.571428571428571|
|       7|Kolkata Knight Ri...|           8.34375|
|       8|     Kings XI Punjab|10.112359550561797|
|       8|Royal Challengers...| 7.219512195121951|
|       9|    Delhi Daredevils|

In [23]:
rr= rr.withColumn("new_column",lit("ABC"))
w1 = Window().partitionBy('new_column').orderBy(lit('A'))
rr = rr.withColumn("row_num", row_number().over(w1)).drop("new_column")

In [24]:
rr.createOrReplaceTempView("rr")

In [25]:
rr.show()

+--------+--------------------+------------------+-------+
|match_id|        batting_team|         Runs_Rate|row_num|
+--------+--------------------+------------------+-------+
|       1| Sunrisers Hyderabad|             9.936|      1|
|       1|Royal Challengers...| 8.390243902439025|      2|
|       2|      Mumbai Indians|             8.832|      3|
|       2|Rising Pune Super...| 9.196721311475411|      4|
|       3|       Gujarat Lions|               9.0|      5|
|       3|Kolkata Knight Ri...|              11.5|      6|
|       4|     Kings XI Punjab| 8.065573770491804|      7|
|       4|Rising Pune Super...| 7.824000000000001|      8|
|       5|Royal Challengers...| 7.596774193548387|      9|
|       5|    Delhi Daredevils| 6.870967741935483|     10|
|       6| Sunrisers Hyderabad|              8.75|     11|
|       6|       Gujarat Lions|              6.75|     12|
|       7|      Mumbai Indians| 8.571428571428571|     13|
|       7|Kolkata Knight Ri...|           8.34375|     1

In [26]:
rr_row_odd = spark.sql('select * from rr where row_num%2 !="0"')
rr_row_even = spark.sql('select * from rr where row_num%2 ="0"')

In [27]:
rr_row_odd.show()
rr_row_even.show()

+--------+--------------------+-----------------+-------+
|match_id|        batting_team|        Runs_Rate|row_num|
+--------+--------------------+-----------------+-------+
|       1| Sunrisers Hyderabad|            9.936|      1|
|       2|      Mumbai Indians|            8.832|      3|
|       3|       Gujarat Lions|              9.0|      5|
|       4|     Kings XI Punjab|8.065573770491804|      7|
|       5|Royal Challengers...|7.596774193548387|      9|
|       6| Sunrisers Hyderabad|             8.75|     11|
|       7|      Mumbai Indians|8.571428571428571|     13|
|       8|Royal Challengers...|7.219512195121951|     15|
|       9|    Delhi Daredevils|             9.84|     17|
|      10|      Mumbai Indians|8.442477876106196|     19|
|      11|Kolkata Knight Ri...|9.865384615384617|     21|
|      12|Royal Challengers...|6.870967741935483|     23|
|      13|Rising Pune Super...|8.274193548387096|     25|
|      14| Sunrisers Hyderabad|              7.5|     27|
|      15|    

In [28]:
rr_row_odd.createOrReplaceTempView("rr_row_odd")
rr_row_even.createOrReplaceTempView("rr_row_even")

# Net Run Rate for each match played by each team

In [29]:
nrr = spark.sql("select rr_row_odd.match_id, rr_row_odd.batting_team as team_1, rr_row_even.batting_team as team_2, (rr_row_odd.Runs_rate-rr_row_even.Runs_rate) as Net_Match_Run_Rate from rr_row_odd join rr_row_even where rr_row_odd.match_id = rr_row_even.match_id")

In [30]:
nrr.show()

+--------+--------------------+--------------------+--------------------+
|match_id|              team_1|              team_2|  Net_Match_Run_Rate|
+--------+--------------------+--------------------+--------------------+
|       1| Sunrisers Hyderabad|Royal Challengers...|  1.5457560975609752|
|       2|      Mumbai Indians|Rising Pune Super...| -0.3647213114754102|
|       3|       Gujarat Lions|Kolkata Knight Ri...|                -2.5|
|       4|     Kings XI Punjab|Rising Pune Super...|  0.2415737704918035|
|       5|Royal Challengers...|    Delhi Daredevils|  0.7258064516129039|
|       6| Sunrisers Hyderabad|       Gujarat Lions|                 2.0|
|       7|      Mumbai Indians|Kolkata Knight Ri...| 0.22767857142857117|
|       8|Royal Challengers...|     Kings XI Punjab| -2.8928473554398453|
|       9|    Delhi Daredevils|Rising Pune Super...|  3.4241584158415836|
|      10|      Mumbai Indians| Sunrisers Hyderabad|  0.9186683522966721|
|      11|Kolkata Knight Ri...|     Ki

In [31]:
nrr.createOrReplaceTempView("nrr")

In [32]:
whole_df = spark.sql("select * from deliveries join matches where deliveries.match_id = matches.id ")

In [33]:
whole_df.createOrReplaceTempView("whole_df")

# Year-wise Run Rate for each team

In [34]:
team_rr = spark.sql("select season, batting_team, (sum(total_runs)/(count(ball)/6)) as Batting_Run_Rate from whole_df group by season, batting_team order by season, batting_team")

In [35]:
team_rr.createOrReplaceTempView("team_rr")

In [36]:
team_rr.show()

+------+--------------------+------------------+
|season|        batting_team|  Batting_Run_Rate|
+------+--------------------+------------------+
|  2008| Chennai Super Kings| 8.068303094983992|
|  2008|     Deccan Chargers| 8.017985611510792|
|  2008|    Delhi Daredevils| 8.214608920491274|
|  2008|     Kings XI Punjab| 8.501437607820588|
|  2008|Kolkata Knight Ri...|7.4836223506743735|
|  2008|      Mumbai Indians| 7.878787878787879|
|  2008|    Rajasthan Royals| 8.385814078452444|
|  2008|Royal Challengers...|7.1717902350813745|
|  2009| Chennai Super Kings| 7.832650672908134|
|  2009|     Deccan Chargers| 7.644444444444445|
|  2009|    Delhi Daredevils| 7.468457943925234|
|  2009|     Kings XI Punjab| 6.857142857142857|
|  2009|Kolkata Knight Ri...| 6.985545335085414|
|  2009|      Mumbai Indians| 7.069565217391305|
|  2009|    Rajasthan Royals|6.6166774402068524|
|  2009|Royal Challengers...| 7.091757387247278|
|  2010| Chennai Super Kings| 8.008200922603793|
|  2010|     Deccan 

In [37]:
opponent_rr = spark.sql("select season, bowling_team, (sum(total_runs)/(count(*)/6)) as Bowler_Run_Rate from whole_df group by season, bowling_team order by season, bowling_team")

In [38]:
opponent_rr.createOrReplaceTempView("opponent_rr")

In [39]:
opponent_rr.show()

+------+--------------------+------------------+
|season|        bowling_team|   Bowler_Run_Rate|
+------+--------------------+------------------+
|  2008| Chennai Super Kings| 8.075471698113208|
|  2008|     Deccan Chargers| 8.373865698729583|
|  2008|    Delhi Daredevils| 8.034939759036144|
|  2008|     Kings XI Punjab|  8.19785189372527|
|  2008|Kolkata Knight Ri...| 7.635555555555555|
|  2008|      Mumbai Indians|7.7295636140135215|
|  2008|    Rajasthan Royals| 7.592417061611374|
|  2008|Royal Challengers...| 8.151571164510166|
|  2009| Chennai Super Kings| 7.394833948339484|
|  2009|     Deccan Chargers|7.2663622526636225|
|  2009|    Delhi Daredevils| 7.177383592017738|
|  2009|     Kings XI Punjab| 7.300645161290323|
|  2009|Kolkata Knight Ri...|7.5864276568501925|
|  2009|      Mumbai Indians| 6.908626198083067|
|  2009|    Rajasthan Royals| 6.775990099009902|
|  2009|Royal Challengers...|7.2677824267782425|
|  2010| Chennai Super Kings| 7.650455927051672|
|  2010|     Deccan 

# Year-wise Net Tournament Run Rate for each team

In [40]:
ntrr = spark.sql("select team_rr.season, team_rr.batting_team, team_rr.Batting_Run_Rate - opponent_rr.Bowler_Run_Rate as Net_Tournament_Run_Rate from team_rr join opponent_rr where team_rr.season = opponent_rr.season and team_rr.batting_team = opponent_rr.bowling_team order by season")

In [41]:
ntrr.show()

+------+--------------------+-----------------------+
|season|        batting_team|Net_Tournament_Run_Rate|
+------+--------------------+-----------------------+
|  2008|     Kings XI Punjab|      0.303585714095318|
|  2008|Royal Challengers...|    -0.9797809294287916|
|  2008|      Mumbai Indians|    0.14922426477435735|
|  2008|    Delhi Daredevils|    0.17966916145513068|
|  2008|     Deccan Chargers|    -0.3558800872187913|
|  2008| Chennai Super Kings|   -0.00716860312921...|
|  2008|    Rajasthan Royals|     0.7933970168410704|
|  2008|Kolkata Knight Ri...|    -0.1519332048811819|
|  2009|Kolkata Knight Ri...|    -0.6008823217647787|
|  2009|Royal Challengers...|   -0.17602503953096438|
|  2009|      Mumbai Indians|    0.16093901930823762|
|  2009|     Kings XI Punjab|   -0.44350230414746594|
|  2009| Chennai Super Kings|     0.4378167245686502|
|  2009|     Deccan Chargers|    0.37808219178082236|
|  2009|    Rajasthan Royals|   -0.15931265880304935|
|  2009|    Delhi Daredevils

In [42]:
ntrr.createOrReplaceTempView("ntrr")

# Teams with positive Net Run Rate over year-wise tournaments

In [43]:
positive_ntrr = spark.sql("select season, batting_team as team, Net_Tournament_Run_Rate from ntrr where Net_Tournament_Run_Rate > 0 order by season asc, Net_Tournament_Run_Rate desc ")

In [44]:
positive_ntrr.show()

+------+--------------------+-----------------------+
|season|                team|Net_Tournament_Run_Rate|
+------+--------------------+-----------------------+
|  2008|    Rajasthan Royals|     0.7933970168410704|
|  2008|     Kings XI Punjab|      0.303585714095318|
|  2008|    Delhi Daredevils|    0.17966916145513068|
|  2008|      Mumbai Indians|    0.14922426477435735|
|  2009| Chennai Super Kings|     0.4378167245686502|
|  2009|     Deccan Chargers|    0.37808219178082236|
|  2009|    Delhi Daredevils|     0.2910743519074961|
|  2009|      Mumbai Indians|    0.16093901930823762|
|  2010|      Mumbai Indians|     0.7088988688639093|
|  2010| Chennai Super Kings|      0.357744995552121|
|  2010|Royal Challengers...|     0.2644201419775696|
|  2010|    Delhi Daredevils|     0.0823491283943385|
|  2011| Chennai Super Kings|     0.5251326644883427|
|  2011|Royal Challengers...|    0.32392426609135505|
|  2011|Kolkata Knight Ri...|    0.22283835061978152|
|  2011|     Deccan Chargers

In [45]:
positive_ntrr.createOrReplaceTempView("positive_ntrr")

# No. of times, teams achieved positive Net run rate for tournaments over the last decade
This stat can help investors choosing the right team to invest their money into.

In [46]:
team_bet = spark.sql("select team, count(*) as num_of_times_had_positive_NRR from positive_ntrr group by team order by count(*) desc")

In [47]:
team_bet.show()

+--------------------+-----------------------------+
|                team|num_of_times_had_positive_NRR|
+--------------------+-----------------------------+
| Chennai Super Kings|                            9|
|      Mumbai Indians|                            9|
|Kolkata Knight Ri...|                            7|
|    Delhi Daredevils|                            6|
|Royal Challengers...|                            5|
| Sunrisers Hyderabad|                            4|
|     Kings XI Punjab|                            3|
|    Rajasthan Royals|                            3|
|     Deccan Chargers|                            2|
|Rising Pune Super...|                            1|
|      Delhi Capitals|                            1|
+--------------------+-----------------------------+

